# Build a simple LLM application with chat models and prompt templates
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPEN_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if langchain_api_key:
    os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# Using Language Models
First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably.

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-342693e0-2673-4194-9a13-9e25a9ab2763-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")]) 

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-835c2ec7-2255-4da7-ac0f-68a5e0bd1161-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Streaming
Because chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation.

In [19]:
for token in model.stream(messages):
    print(token.content, end="|")

|C|iao|!||

# Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [17]:
prompt = prompt_template.invoke({"language": "Spanish", "text": "Bye!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='Bye!', additional_kwargs={}, response_metadata={})])

In [20]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Bye!', additional_kwargs={}, response_metadata={})]

In [21]:
response = model.invoke(prompt)
print(response.content)

¡Adiós!
